In [1]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
#from accelerate import Accelerator
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
def load_dataset(dataset_dir):
  dataset_dir = Path(dataset_dir)
  data = []
  label_data = []

  for label_file in ['pos', 'neg']:
    for data_file in (dataset_dir/label_file).iterdir():
      data.append(data_file.read_text(encoding='utf-8'))
      label = 1 if label_file == 'pos' else 0
      label_data.append(label)

  return data, label_data

In [3]:
train_data, train_label = load_dataset('/content/drive/MyDrive/custom-model/aclImdb/train')
test_data, test_label = load_dataset('/content/drive/MyDrive/custom-model/aclImdb/test')


In [4]:
train_data, val_data, train_label, val_label = train_test_split(train_data, train_label, test_size=0.2)


In [5]:
model = "distilbert-base-uncased"

tokenizer = DistilBertTokenizerFast.from_pretrained(model)

train_encode = tokenizer(train_data, truncation=True, padding=True)
val_encode = tokenizer(val_data, truncation=True, padding=True)
test_encode = tokenizer(test_data, truncation=True, padding=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [6]:
class CustomDataset(Dataset):

  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])

    return item


  def __len__(self):
    return len(self.labels)

In [7]:
train_dataset = CustomDataset(train_encode, train_label)
val_dataset = CustomDataset(val_encode, val_label)
test_dataset = CustomDataset(test_encode, test_label)


In [8]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  accuracy = accuracy_score(labels, preds)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
  return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
  }

In [9]:
training_arg = TrainingArguments(
    output_dir='/content/drive/MyDrive/sentiment_analysis_model/results',
    num_train_epochs=2,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=70,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/sentiment_analysis_model/logs',
    logging_steps=10,
    warmup_steps=500,

)

In [10]:
Bert_model = DistilBertForSequenceClassification.from_pretrained(model)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        labels = labels.long()
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss()  #CrossEntropyLoss for multi-class
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [12]:
trainer = Trainer(
    model=Bert_model,
    args=training_arg,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(eval_results)

Step,Training Loss
10,0.695500
20,0.692000
30,0.690200
40,0.694700
50,0.683900
60,0.683000
70,0.677600
80,0.671400
90,0.651100
100,0.605100


{'eval_loss': 0.22045594453811646, 'eval_accuracy': 0.92872, 'eval_precision': 0.9292091858473757, 'eval_recall': 0.92872, 'eval_f1': 0.9286996840872657, 'eval_runtime': 383.4421, 'eval_samples_per_second': 65.199, 'eval_steps_per_second': 0.934, 'epoch': 2.0}
